In [1]:
import numpy as np
import os

from keras.src.callbacks import CSVLogger
from keras.src.layers import Masking
import time
import mediapipe as mp

2024-06-12 20:38:41.618892: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-12 20:38:41.702514: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-12 20:38:41.946316: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 20:38:45.586569: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
sequence_length = 30  # Number of frames per sequence
shape_no_face = (sequence_length, 258)
num_classes = 4
actions = ['hallo','du','idle','a']  # List all your actions including idle
DATA_PATH = 'CollectionBase/MP_Data/30frame_no_face_+idle'  # Update with your data path
landmarks = [False,True,True,True] # Select True/False for different landmarks (face, pose, left hand, right hand)

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def extract_keypoints(results):
    arr = []
    if(landmarks[0]):
        face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
        arr.append(face)
    if(landmarks[1]):
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
        arr.append(pose)
    if(landmarks[2]):
        lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
        arr.append(lh)
    if(landmarks[3]):
        rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
        arr.append(rh)
    return np.concatenate(arr)

def draw_landmarks(image, results):
    if(landmarks[0]):
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    if(landmarks[1]):
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    if(landmarks[2]):
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    if(landmarks[3]):
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)


def draw_styled_landmarks(image, results):
    if(landmarks[0]):
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                  mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                  )
    if(landmarks[1]):
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                  )
    if(landmarks[2]):
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                  )
    if(landmarks[3]):
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

In [4]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Paths and parameters

# Initialize sequences and labels
sequences, labels = [], []

# Create a mapping of labels to numbers
label_map = {label: num for num, label in enumerate(actions)}

# Loop through each action and each sequence within that action
for action in actions:
    action_dir = os.path.join(DATA_PATH, action)
    for sequence in np.array(os.listdir(action_dir)).astype(int):
        window = []
        for frame_num in range(sequence_length):
            frame_path = os.path.join(action_dir, str(sequence), f"{frame_num}.npy")
            res = np.load(frame_path)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

# Convert sequences and labels to numpy arrays
X = np.array(sequences)
y = to_categorical(labels).astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# Print the shape of the data
print(f"Total sequences: {len(sequences)}")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

import numpy as np

# Save the arrays to .npy files
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

Total sequences: 120
X shape: (120, 30, 258)
y shape: (120, 4)
X_train shape: (114, 30, 258), y_train shape: (114, 4)
X_test shape: (6, 30, 258), y_test shape: (6, 4)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
import os

# Define the L2 regularization factor
l2_factor = 0.00
dropout = 0.2

model = Sequential()

# Input layer
model.add(Masking(mask_value=0., input_shape=shape_no_face))  # Adjust input shape accordingly

# Adding LSTM layers with L2 regularization
model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(LSTM(128, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(dropout))
model.add(BatchNormalization())

# Fully connected layer with L2 regularization
model.add(Dense(256, activation='relu', kernel_regularizer=l2(l2_factor)))
model.add(Dropout(dropout))

# Output layer for num_classes with L2 regularization
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=l2(l2_factor)))

# Set up CSVLogger
csv_logger = CSVLogger('training_log.csv', append=True)

# Set up TensorBoard
log_dir = os.path.join("logs", "fit", "sign_language_recognition")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# List of callbacks
callbacks = [csv_logger, tensorboard_callback, early_stopping]

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=callbacks)

#history = model.fit(X_train, y_train,
#                    epochs=10,
#                    batch_size=32,
#                    validation_data=(X_test, y_test),
#                    callbacks=callbacks)
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

In [ ]:
# Register custom objects
# custom_objects = {'CustomLayer': CustomLayer, ...}

# Save the model with custom objects
tf.saved_model.save(model, './model')
